In [1]:
import torch
import torch.nn as nn

def psuedo_quant_tensor(w, n_bit=4, group_size=128):
    """
    실제 int4 로 변환하지 않고, float16 상태에서 값만 4bit 스타일로 뭉개는 함수
    PPL 을 빠르게 예측하는 데 유리
    """
    org_w_shape = w.shape

    # 그룹 단위로 reshape

    # scale 계산
    # 각 채널 별 절댓값 최대치 찾기
    max_val = w.abs().amax(dim=1, keepdim=True)
    max_val = max_val.clamp(min=1e-5)

    # 4bit 의 최대 표현 범위 지정
    max_int = 2 ** (n_bit - 1) - 1
    min_int = - (2 ** (n_bit - 1))
    scale = max_val / max_int

    w_quant = torch.clamp(torch.round(w / scale), min=min_int, max=max_int)

    # 역양자화 과정 (다시 복원해보면 정보 손실을 확인할 수 있음)
    w_dequant = w_quant * scale

    return w_dequant, scale

# 테스트
dummy_weight = torch.randn(10, 10, dtype=torch.float16)
q_weight, s = psuedo_quant_tensor(dummy_weight)

print(f"원본 일부: {dummy_weight[0][:5]}")
print(f"양자화 적용본: {q_weight[0][:5]}")
print(f"오차(MSE): {(dummy_weight - q_weight).pow(2).mean().item()}")

원본 일부: tensor([-0.2172,  0.6953,  0.0411,  2.3066, -0.1843], dtype=torch.float16)
양자화 적용본: tensor([-0.3296,  0.6592,  0.0000,  2.3066, -0.3296], dtype=torch.float16)
오차(MSE): 0.007106781005859375


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# 모델 로드 (EXAONE 4.0 1.2B)
model_id = "LGAI-EXAONE/EXAONE-4.0-1.2B"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 샘플 데이터 투입
input_text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)


def layer_sensitivity(model, inputs):
    """
    모델 레이어의 양자화 민감도를 측정하는 함수
    """
    sensitivities = {}
    print("모델 레이어별 민감도 측정을 시작합니다.")

    layers = model.model.layers

    for i, layer in enumerate(tqdm(layers)):

        # 1. 원본 레이어의 출력값 뽑아내기
        